In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
import pandas as pd
import numpy as np
from itertools import cycle

In [ ]:
#| export
def foo(): pass

In [ ]:
#|export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("water")

'Hello water!'

## test index

In [ ]:
from IPython.display import display,SVG

In [ ]:
display(SVG('<svg height="100" xmlns="http://www.w3.org/2000/svg"><circle cx="50" cy="50" r="40"/></svg>'))

In [ ]:
assert say_hello("Hamel")=="Hello Hamel!"

In [ ]:
say_hello("Isaac")

'Hello Isaac!'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

## test dataframe


In [ ]:
#|export

def generate_fake_dataframe(size, cols, col_names = None, intervals = None, seed = None):
    'generate long fake dataframe'
    
    categories_dict = {'animals': ['cow', 'rabbit', 'duck', 'shrimp', 'pig', 'goat', 'crab', 'deer', 'bee', 'sheep', 'fish', 'turkey', 'dove', 'chicken', 'horse'],
                       'names'  : ['James', 'Mary', 'Robert', 'Patricia', 'John', 'Jennifer', 'Michael', 'Linda', 'William', 'Elizabeth', 'Ahmed', 'Barbara', 'Richard', 'Susan', 'Salomon', 'Juan Luis'],
                       'cities' : ['Stockholm', 'Denver', 'Moscow', 'Marseille', 'Palermo', 'Tokyo', 'Lisbon', 'Oslo', 'Nairobi', 'Río de Janeiro', 'Berlin', 'Bogotá', 'Manila', 'Madrid', 'Milwaukee'],
                       'colors' : ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'purple', 'pink', 'silver', 'gold', 'beige', 'brown', 'grey', 'black', 'white']
                      }
    default_intervals = {"i" : (0,10), "f" : (0,100), "c" : ("names", 5), "d" : ("2020-01-01","2020-12-31")}
    rng = np.random.default_rng(seed)

    first_c = default_intervals["c"][0]
    categories_names = cycle([first_c] + [c for c in categories_dict.keys() if c != first_c])
    default_intervals["c"] = (categories_names, default_intervals["c"][1])
    
    if isinstance(col_names,list):
        assert len(col_names) == len(cols), f"The fake DataFrame should have {len(cols)} columns but col_names is a list with {len(col_names)} elements"
    elif col_names is None:
        suffix = {"c" : "cat", "i" : "int", "f" : "float", "d" : "date"}
        col_names = [f"column_{str(i)}_{suffix.get(col)}" for i, col in enumerate(cols)]

    if isinstance(intervals,list):
        assert len(intervals) == len(cols), f"The fake DataFrame should have {len(cols)} columns but intervals is a list with {len(intervals)} elements"
    else:
        if isinstance(intervals,dict):
            assert len(set(intervals.keys()) - set(default_intervals.keys())) == 0, f"The intervals parameter has invalid keys"
            default_intervals.update(intervals)
        intervals = [default_intervals[col] for col in cols]
    df = pd.DataFrame()
    for col, col_name, interval in zip(cols, col_names, intervals):
        if interval is None:
            interval = default_intervals[col]
        assert (len(interval) == 2 and isinstance(interval, tuple)) or isinstance(interval, list), f"This interval {interval} is neither a tuple of two elements nor a list of strings."
        if col in ("i","f","d"):
            start, end = interval
        if col == "i":
            df[col_name] = rng.integers(start, end, size)
        elif col == "f":
            df[col_name] = rng.uniform(start, end, size)
        elif col == "c":
            if isinstance(interval, list):
                categories = np.array(interval)
            else:
                cat_family, length = interval
                if isinstance(cat_family, cycle):
                    cat_family = next(cat_family)
                assert cat_family in categories_dict.keys(), f"There are no samples for category '{cat_family}'. Consider passing a list of samples or use one of the available categories: {categories_dict.keys()}"
                categories = rng.choice(categories_dict[cat_family], length, replace = False, shuffle = True)
            df[col_name] = rng.choice(categories, size, shuffle = True)
        elif col == "d":
            df[col_name] = rng.choice(pd.date_range(start, end), size)
    return df 

In [ ]:
generate_fake_dataframe(size = 1000, cols =  "cififficcdccc")

,column_0_cat,column_1_int,column_2_float,column_3_int,column_4_float,column_5_float,column_6_int,column_7_cat,column_8_cat,column_9_date,column_10_cat,column_11_cat,column_12_cat
0,John,9,64.685800,1,26.230901,57.968733,2,deer,Oslo,2020-03-23,blue,Robert,sheep
1,John,9,16.432777,1,53.610216,47.676311,8,pig,Nairobi,2020-10-08,green,Robert,chicken
2,Barbara,9,8.892160,8,34.800453,18.564650,4,bee,Tokyo,2020-07-20,beige,William,sheep
3,Elizabeth,6,48.260249,5,62.355616,86.320202,3,shrimp,Tokyo,2020-07-26,yellow,William,turkey
4,Michael,8,97.392517,5,54.817999,13.179147,3,shrimp,Oslo,2020-07-09,blue,Richard,sheep
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Barbara,0,52.177797,2,80.179861,25.327817,6,shrimp,Tokyo,2020-05-25,yellow,Robert,deer
996,Michael,6,20.014674,4,88.916244,6.589388,0,deer,Nairobi,2020-03-21,yellow,James,chicken
997,Barbara,5,26.575786,8,9.459123,56.085015,3,duck,Oslo,2020-10-31,gold,Robert,chicken
998,Michael,1,98.383791,8,38.970918,0.813904,8,deer,Tokyo,2020-01-31,beige,Jennifer,deer
